
<a target="_blank" href="https://colab.research.google.com/github/taobrienlbl/advanced_earth_science_data_analysis/blob/spring_2023_iub/lessons/06_advanced_plotting/06_workalong02_mapping.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Mapping

This workalong and exercise introduce mapping in `matplotlib`. We'll build from the figure we created in the first workalong.

In [ ]:
""" Check if this notebook is being run in Google Colab """

# if the notebook is being run in google colab, we need to install cartopy in a special way

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


if IN_COLAB:
  # install cartopy if it isn't already installed
  try:
    import cartopy
  except:
    # force re-installation of shapely, otherwise cartopy breaks in google colab
    # upgrade pip
    ! pip install --upgrade pip
    # remove the current version of shapely
    ! pip uninstall shapely --yes
    # install shapely and cartopy simultaneously, forcing compilation
    ! pip install --no-binary shapely cartopy
    # install cmocean too
    ! pip install cmocean

In [ ]:
""" Import libraries """


In [ ]:
""" Load the dataset """

input_file = "air.2m.gauss.2021.nc"

# download the data file if needed
import os
if not os.path.exists(input_file):
    url = f"https://psl.noaa.gov/thredds/fileServer/Datasets/ncep.reanalysis2/gaussian_grid/air.2m.gauss.2021.nc"
    ! curl --output {input_file} {url}

# open the dataset using xarray
temp_ds = xr.open_dataset(input_file, chunks = -1)

""" Calculate the hottest temperature recorded in each season. """

# xarray is built on top of pandas, so we can use groupby for this
season_groups = temp_ds.groupby('time.season')

# calculate the max in each season
season_max_temp = season_groups.max()

# extract temperature (and also use 'squeeze' to remove the pesky single-item level dimension)
max_temp_xr = season_max_temp['air'].squeeze()

# pull out coordinates
seasons = max_temp_xr.season
lat = max_temp_xr.lat
lon = max_temp_xr.lon

# force the calculation
max_temp_xr.load();

# Cartopy

For mapping, we'll use `cartopy`.  Cartopy defines *projection* as the map projection on which the dataset will be drawn.  It defines the *transform* of the dataset as the projection on which the dataset exists.  They can be the same, but they don't have to be.

Let's just explore projections.

In [ ]:
""" Plot the hottest temperature recorded in each season with maplines. """
